# 朴素贝叶斯

（1）基于朴素贝叶斯理论的分类方法

    优点：在数据较少的情况下，可以处理多类别问题
    缺点：对于输入数据的准备方式较为敏感
    适用数据类型：标称型数据

（2）贝叶斯决策理论的核心思想：选择具有最高概率的决策

    假设 p1(x,y)表示数据点(x,y)属于类别1，p2(x,y)表示数据点属于类别2
    如果 p1(x,y)>p2(x,y)，那么类别为1
    如果 p2(x,y)>p1(x,y)，那么类别为2
    
        这里p1(),p2()是为了简述，实际上为 p(c1|x,y),p(c2|x,y)(条件概率):意思为给点某个由想x，y表示的数据点，那么该数据点来着类别ci的概率是多少
                             p(c|x,y)=p(x,y|c)*p(c)/p(x,y)
        使用这些定义，可以定义朴素贝叶斯准则为：
        如果 p(c1|x,y)>p(c2|x,y)，那么属于类别c1
        如果 p(c2|x,y)>p(c1|x,y)，那么属于类别c2

（3）朴素贝叶斯的一般过程：

    （1）收集数据：可以使用任何方法。本章使用RSS源
    （2）准备数据：需要数值型或者布尔型数据
    （3）分析数据：有大量特征时，绘制特征作用不大，此时使用直方图效果更好
    （4）训练算法：计算不同的独立特征的条件概率
    （5）测试算法：计算错误率
    （6）使用算法：一个常见的朴素贝叶斯应该是文档分类。可以在任意的分类场景中使用朴素贝叶斯分类器，不一定是文本

（4）朴素贝叶斯的两个假设：

    （1）特征之间相互独立
    （2）每个特征同等重要

    例子：根据社区留言板上的单词，判断该条留言为侮辱类还是非侮辱类，分别用1和0表示

3.1 准备数据，从文本中构建单词向量

In [1]:
#词表到向量的转换函数
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] #1代表侮辱性文字，0代表正常言论
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet|set(document) #求并集
    return list(vocabSet)
 
#判断给定句子中的单词是否在单词表中出现
def setOfWords2Vec(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word %s is not in my Vocabulary!"%word)
    return returnVec

In [2]:
postingList,classVec = loadDataSet()
print(postingList)
print(classVec)
myVocabList= createVocabList(postingList)
print(myVocabList)
returnVec = setOfWords2Vec(myVocabList,postingList[0])
print(returnVec)

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
[0, 1, 0, 1, 0, 1]
['flea', 'take', 'to', 'garbage', 'help', 'not', 'how', 'problems', 'quit', 'please', 'maybe', 'I', 'licks', 'buying', 'food', 'him', 'is', 'park', 'stupid', 'ate', 'love', 'cute', 'worthless', 'posting', 'mr', 'has', 'my', 'steak', 'stop', 'so', 'dalmation', 'dog']
[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1]


3.2 训练算法：从词向量计算概率

    1.利用朴素贝叶斯进行分类时，要计算多个概率乘积获得文档属于哪个类别。即计算p(w0|ci)P(w1|ci)p(w2|ci)...。如果其中一个概率值为0，最后结果也为0，为避免出现这种影响。我们将分子初始化为1，分母初始化为2
    2.另一个问题是下溢出，原因是因为很多很小的数相乘。所以，这里我们选择取对数，并且有 In(a*b) = Ina+Inb

In [3]:

from numpy import *
#trainMatrix:文档矩阵，trainCategory:由每篇文档类别标签所构成的向量

def trainNB(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)      #change to np.ones()
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)          #change to np.log()
    p0Vect = log(p0Num/p0Denom)          #change to np.log()
    return p0Vect, p1Vect, pAbusive

In [4]:
postingList,classVec = loadDataSet()
myVocabList= createVocabList(postingList)
trainMat = []
for postinDoc in postingList:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
print(trainMat)
p0Vect,p1Vect,pAbusive = trainNB(trainMat,classVec)
print(p0Vect)
print(p1Vect)
print(pAbusive)


[[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[-2.56494936 -3.25809654 -2.56494936 -3.25809654 -2.56494936 -3.25809654
 -2.56494936 -2.56494936 -3.25809654 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -3.25809654 -3.25809654 -2.15948425 -2.56494936 -3.25809654
 -3.25809654 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -3.25809654
 -2.56494936 -2.56494936 -1.87180218 -2.56494936 -2.56494936 -2.56494936
 -2.56494936 -2.56494936]
[-3.04452244 -2.3513

3.3 测试算法：根据现实情况修改分类器

朴素贝叶斯分类函数

In [5]:
#vec2Classify要分类向量
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    #根据朴素贝叶斯分类函数分别计算待分类文档属于类1和类0的概率
    p1=sum(vec2Classify*p1Vec)+log(pClass1)
    p0=sum(vec2Classify*p0Vec)+log(1.0-pClass1)
    if p1>p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabLlist = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB(array(trainMat),array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as :',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as :',classifyNB(thisDoc,p0V,p1V,pAb))

In [6]:
testingNB()

['love', 'my', 'dalmation'] classified as : 0
['stupid', 'garbage'] classified as : 1


3.4 准备模型：文档词袋模型

    之前我们将每个词出现的是否出现作为特征，这可以被描述为词集模型。
    如果一个词在文档中出现不止一次，这可能意味着包含该词是否出现在文档中所不能到达的某种信息，这种方法称为词袋模型。
    在词袋中每个单词可以出现多次，而词集中每个词只能出现一次

In [7]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

3.5 使用朴素贝叶斯进行垃圾分类

    （1）收集数据：提供文本文件
    （2）准备数据：将文本文件解析为词条向量
    （3）分析数据：检查词条确保解析的正确性
    （4）训练算法：使用之前建立的trainNB0（）函数
    （5）测试算法：使用classifyNB()，并且构建一个新的测试函数来计算文档集的错误率
    （6）使用算法：构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上。

测试算法：使用朴素贝叶斯进行交叉验证

In [8]:
#切分文本
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W+',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]
    listOfTokens = re.split(r'\W+',bigString)

In [9]:
def spamTest():
    docList = []; classList = []; fullText = []
    for i in range(1, 26):
        wordList = textParse(open('D:\\AGAME\\MachineLearning\\email\\spam/%d.txt'%i,encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('D:\\AGAME\\MachineLearning\\email\\ham/%d.txt'%i,encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = range(50); testSet = []           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(list(trainingSet)[randIndex])
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error", docList[docIndex])
    print('the error rate is: ', float(errorCount)/len(testSet))

In [10]:
spamTest()

the error rate is:  0.0
